In [ ]:
# %load notebook_setup.py
#Use %load notebook_setup.py to load these commands at the top of your
#ipython notebook.  Includes running the std_libraries.py file. 

%run std_libraries.py
%load_ext autoreload
%autoreload 2
# %matplotlib 
# plt.ioff()
%matplotlib inline
#matplotlib notebook

y1000_plus_dir_ubuntu = '/home/heineike/GitHub/y1000plus_tools/data/'

In [ ]:
y1000_species = pd.read_csv(y1000plus_dir + os.path.normpath("y1000plus_tools_data/y1000plus/y1000_species_table.csv"), index_col=0)

#Select only clades of interest: 
#clades_of_interest = {'Saccharomycetaceae','Saccharomycodaceae','Phaffomycetaceae','CUG-Ser2','CUG-Ser1','Pichiaceae','CUG-Ala'}
#clades_of_interest = {'Saccharomycetaceae', 'Saccharomycodaceae'}
#outgroups = [ 'Wickerhamomyces anomalus','Cyberlindnera jadinii', 'Ascoidea rubescens','Komagataella pastoris','Debaryomyces hansenii', 'Candida albicans', 'Yarrowia lipolytica']
#outgroup = 'Yarrowia lipolytica'

#y1000_species_subset = y1000_species[(y1000_species['Major clade'].isin(clades_of_interest))  |  (y1000_species['Species name'].isin(outgroups))]


#outgroup = 'Hanseniaspora vineae'  #For EGO2/4 - only Sacch so use different outgroup
#y1000_species_subset = pd.concat([y1000_species_subset, y1000_species[y1000_species['Species name']==outgroup]])

y1000_species_subset = y1000_species[y1000_species['Major clade']!='outgroup']
#y1000_species_subset = y1000_species​


In [ ]:
#Load S.Cer lookup table: 
scer_lookup_fname = y1000plus_dir + os.path.normpath('y1000plus_tools_data/y1000plus/id_lookups/saccharomyces_cerevisiae.csv')

scer_lookup = pd.read_csv(scer_lookup_fname, index_col=0)

#Met17: 'YLR303W', 
#Str2:  YJR130C
#'YML082W', 
#Str3: 'YGL184C', 
#'YLL058W', 
#Cys3: 'YAL012W'
#IRC7: 'YFR055W'


target_y1000_ids = scer_lookup.loc[['YLR303W', 'YML082W','YJR130C', 'YFR055W', 'YHR112C', 'YGL184C', 'YLL058W', 'YAL012W'], 'y1000_id']

y1000_id_list = list(target_y1000_ids)
goi_og_lookup, og_genes_lookup = y1000plus_tools.make_og_genes_lookup(y1000_id_list, y1000_species_subset)

In [ ]:
##Only need to do once

# for protein_fname in ['IRC7', 'YHR112C']:
#     target_y1000_id = protein_fnames[protein_fname]
#     og_genes = og_genes_lookup[goi_og_lookup[target_y1000_id]]
#     y1000plus_tools.extract_protein_seqs(og_genes, protein_fname, y1000_species_subset)
#     print(protein_fname + ' protein sequence extraction complete')

In [ ]:
goi = 'YHR112C'  #'YHR112C' #'STR3'  #YHR112C, IRC7
 
protein_fname_base = y1000_plus_dir_ubuntu + 'y1000plus_tools_data/y1000plus/proteins_og/' + goi + '/' + goi

#Command:   mafft v 7.487
mafft_command = ['mafft', '--genafpair', '--maxiterate', '1000', 
                protein_fname_base + '.fasta', 
                '>',
                protein_fname_base + '_aln.fasta']


print(" ".join(mafft_command))


#Trimming Command: 
#trimal 1.2r59
#Had to install g++ on the linux system to make the program.  
trimal_command = ['/home/heineike/trimAl/source/trimal',
                  '-in', protein_fname_base + '_aln.fasta', 
                  '-out', protein_fname_base + '_aln_trimmed.fasta', 
                  '-gappyout']

print("\nTrimal command\n")
print(" ".join(trimal_command))


In [ ]:
#Check to see which major clades / species are present in the orthogroup if 
#there is no good outgroup
og_genes = og_genes_lookup[goi_og_lookup[protein_fnames[goi]]]
specs_in_og = []
for gene in og_genes: 
    specs_in_og.append(gene.split('_')[0])

set(y1000_species_subset[y1000_species_subset['spec_og_id'].isin(set(specs_in_og))]['Major clade'])


In [ ]:
#choose outgroup species (important for making protein trees from multiple sequence alignments) 
outgroup = 'Lipomyces starkeyi'#'Yarrowia lipolytica' #'Hanseniaspora vineae'#'Yarrowia lipolytica' #'Lipomyces starkeyi' # 'Tortispora starmeri' #'Lipomyces starkeyi'   #Lipomyces starkeyi is a good outgroup for all the species

outgroup_orig_genome = y1000_species[y1000_species['Species name']==outgroup]['original_genome_id'].values[0]

#Test to see if orthogroup has something from outgroup: 

y1000_id_outgroup = y1000_species_subset[y1000_species_subset['Species name']==outgroup]['spec_og_id'].values[0]

outgroup_gene = ''
for gene in og_genes:#og_genes: 
    y1000_spec_id = gene.split('_')[0]
    if y1000_spec_id == str(y1000_id_outgroup):
        outgroup_gene_id = gene
        
outgrp_orig_spec_name = y1000_species_subset[y1000_species_subset['Species name']==outgroup]['original_genome_id'].values[0]
outgrp_lookup_fname = y1000plus_dir + os.path.normpath('y1000plus_tools_data/y1000plus/id_lookups/' + outgrp_orig_spec_name + '.csv')

outgrp_lookup = pd.read_csv(outgrp_lookup_fname, index_col=0)
y1000_species_subset[y1000_species_subset['Species name']==outgroup]['original_genome_id'].values[0]
outgroup_gene = outgrp_orig_spec_name  + '_' + outgrp_lookup[outgrp_lookup['y1000_id']==outgroup_gene_id].index[0]
    

if outgroup_gene =='': 
    print(outgroup + ' has no proteins in og ' + ogs + ' for ' + goi + '. Pick another outgroup')
else: 
    print('outgroup gene id = ' + outgroup_gene_id)
    print('outgroup gene = ' + outgroup_gene)


In [ ]:

#protein_fname_aln = protein_fname_base + '_aln.fasta'
#prot_seqs = SeqIO.parse(protein_fname_aln, "fasta")  

outgroup_id = outgroup_dict[goi]

# #routine for figuring out outgroup_id if we don't use the outgroup_dict
# if outgroup_gene == None: 
#     print('No outgroup gene designated - need to pick outgroup gene')

# for seq in prot_seqs: 
#     y1000_id = seq.description.split(' ')[1].split('=')[1]
#     if y1000_id == outgroup_gene:
#         outgroup_id = seq.id

#iqtree-1.6.12 linux 64 bit
iqtree_command = ["/home/heineike/iqtree-1.6.12-Linux/bin/iqtree", 
                  "-s" , protein_fname_base + '_aln_trimmed.fasta',
                  "-m", 'LG+R8',  #The first two were LG+R7 and LG+R8, so used LG+R8 to save time
                  "-nt", "AUTO",  #automatically determines number of threads 
                  "-o", outgroup_id]
print(" ".join(iqtree_command))

In [ ]:
#protein_fname_aln = protein_fname_base + '_aln.fasta'
#prot_seqs = SeqIO.parse(protein_fname_aln, "fasta")  

outgroup_id = outgroup_dict[goi]

# #routine for figuring out outgroup_id if we don't use the outgroup_dict
# if outgroup_gene == None: 
#     print('No outgroup gene designated - need to pick outgroup gene')

# for seq in prot_seqs: 
#     y1000_id = seq.description.split(' ')[1].split('=')[1]
#     if y1000_id == outgroup_gene:
#         outgroup_id = seq.id

#iqtree-1.6.12 linux 64 bit
iqtree_command = ["/home/heineike/iqtree-1.6.12-Linux/bin/iqtree", 
                  "-s" , protein_fname_base + '_aln_trimmed.fasta',
                  "-m", 'LG+R8',  #The first two were LG+R7 and LG+R8, so used LG+R8 to save time
                  "-nt", "AUTO",  #automatically determines number of threads 
                  "-o", outgroup_id]
print(" ".join(iqtree_command))